In [1]:
# Nativos
import random as rn
import os
import sys
import gc

#calculo
import numpy as np
import pandas as pd
import scipy

#grafico
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
%matplotlib inline
sns.set(style="whitegrid")

#warning ignore future
import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
gc.collect()

SEED = 29082013
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
rn.seed(SEED)

In [2]:
data = pd.read_csv('AVANCE_6.csv', dtype={
    'COD_UNICO': str,
    'PERIODO': str
})
data.head()

,_0.3_cut_up_APLANCAMIENTO_PATRIMONIAL_VAR_CEF_SITUACION.1,TIENE_NRO_DOCS_IMPAG_REGISTRADO_MES_ACTUAL,_aux_TRX_CAIDA_50PERCENT_TRANSFERENCIAS__U6M,_3.5_cut_up_DTOTALCAJA_S_EBITDA_VAR_CEF_SITUACION,AFRICA_IMP_ADEX,VARACION_NUM_BANCOS_FULL_RCCC_SF,_7_cut_up_ATRASO_PROM_U12M_COLDIRECT_VIG_VENC_SF,GRUPO_ECON_TIENE_TENDENCIA_AUMENTO_CLASIF_NO_NORMAL_U3M,TENDENCIA_A_DISMINUIR_NRO_TRABAJADORES_SUNAT_U7M,_aux_TRX_CAIDA_40PERCENT_OPERACIONES_MESA__U6M,...,_0.99_cut_up_PERCENT_COL_DIRECTAS_SF,_0.7_cut_up_PERCENT_COL_DIRECTAS_SF,_6_cut_up_NUM_BANCOS_NOIBK_DEBT_MENOR_DE_U3M,_0.4_cut_down_minimo_percent_auto_hipo_u12m,_2_cut_up_NUM_VECES_REACTIVA_IBK,ACTIVIDAD_preocupa,_0.5_cut_up_minimo_percent_others_u12m,_20percent_down_COVER_GAR_AUTO_HIPO_PREN_NOPREFER_ULT_RCC,_1_cut_up_TRX_TIPOS_CASH_OUT_CAIDA_10PERCENT_MES_ANT,_-1_cut_down_DIFF_PROM_CANT_TRX_U6M_U12M_CASHIN
0,0.0,0.0,0.0,0.0,NaN,1.0,0,0.0,1.0,0.0,...,0.0,1.0,0,0.0,0,0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,NaN,0.0,0,0.0,1.0,0.0,...,1.0,1.0,0,0.0,0,1,1.0,0.0,0.0,0.0
2,1.0,1.0,0.0,0.0,0.0,-1.0,0,0.0,1.0,0.0,...,0.0,1.0,0,0.0,0,0,1.0,1.0,1.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,1.0,1.0,...,0.0,1.0,0,1.0,0,0,1.0,1.0,1.0,1.0
4,0.0,1.0,0.0,0.0,NaN,0.0,1,0.0,0.0,1.0,...,0.0,1.0,0,1.0,0,0,1.0,1.0,1.0,1.0


In [3]:
from utils_alertas import *

print(data.shape)
data = filtro_banca_s2provincia(data)
print(data.shape)

col_target = 'target_FINAL_fusion_atraso_feve_2Mbueno'

pd.crosstab(
    data[col_target].fillna(-1), data['PERIODO']
)

(23449, 2415)
(1749, 2415)


PERIODO,202009,202010,202011,202012,202101
target_FINAL_fusion_atraso_feve_2Mbueno,,,,,
-1.0,136,128,129,120,98
0.0,192,206,207,213,218
1.0,31,21,16,17,17


In [4]:
data.shape, data[data[col_target].notnull()].shape

data = data[data[col_target].notnull()]

print(data.shape)

(1138, 2415)


In [5]:
data = data[data['PERIODO'].isin(sorted(data['PERIODO'].unique())[-3:])]
data['PERIODO'].value_counts(dropna=False)

202101    235
202012    230
202011    223
Name: PERIODO, dtype: int64

In [6]:
malos = data[
    (data[col_target] == 1 ) #\
    #   & (setalertas['TIENE_DEUDA_REFINANCIADO_RCC'] == 0) & (setalertas['TIENE_DEUDA_REFINANCIADO_IBK'] == 0) & \
    #(setalertas['_3_cut_up_NUM_PROD_DEUDA_VIGENTE_SF_ULT_RCC'] == 0)
]
total_malos = len(
    set(malos['COD_UNICO'])
)
total_cartera = len(
    set(
        data['COD_UNICO']
    )
)

print("malos detectados ", total_malos, " total cartera", total_cartera)

malos detectados  36  total cartera 267


#### COMBINACIONES DE 3 ---- 3 COSECHAS

In [7]:
_vars = pd.read_csv('COMBINACIONES_S1_PROVINCIA.csv').sort_values(by=['SCORE'], ascending=False)
_vars['ALL_VARS'] = _vars['VAR_1'] + _vars['VAR_2'] + _vars['VAR_3']

print(_vars.shape)
_vars.head()

(1465, 7)


,VAR_1,VAR_2,VAR_3,CASOS,CORRECTOS,SCORE,ALL_VARS
254,_1_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASOSF_UL...,_aux_CASHOUT_CAIDA_40PERCENT_PAGOS_MASIVOS__U6M,NaN,7,7,1.000000,NaN
1416,_aux_NO_TIENE_TRX_POS__HACE_12M,_1_cut_up_MAX_DIAS_ATRASO,IMPORTE_IMPAG_REGISTRADO_MAYOR_A_5MIL_SOLES,6,5,0.833333,_aux_NO_TIENE_TRX_POS__HACE_12M_1_cut_up_MAX_D...
247,_4_cut_up_MAX_DIAS_ATRASO_SFSF_ULT_RCC,TIENE_EN_OTRAENTIDAD_DEBT_DESCUENTOS_VIGENTE_S...,NaN,6,5,0.833333,NaN
347,_1_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASOSF_UL...,TIENE_AUMENTO_RIESGO_30PERCENT_A_MAS_U3M,NaN,6,5,0.833333,NaN
62,_-0.1_cut_down_VENTAS_NETAS_VAR_CEF_SITUACION,TIENE_AUMENTO_COL_INDIRECTA_1_PERCENT_A_MAS_U7M,_1_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASOSF_UL...,6,5,0.833333,_-0.1_cut_down_VENTAS_NETAS_VAR_CEF_SITUACIONT...


In [8]:
select = _vars[
    (~_vars['ALL_VARS'].astype(str).str.contains('COVID')) & \
    (~_vars['ALL_VARS'].astype(str).str.contains('ACTIVIDAD')) &\
    (~_vars['ALL_VARS'].astype(str).str.contains('VENCID')) & (~_vars['ALL_VARS'].astype(str).str.contains('REFI')) & \
    (_vars['SCORE'] >= 0.1)  & (_vars['CASOS'] >= 1) &  (_vars['CASOS'] <= 200)
].sort_values(by=['SCORE'], ascending=False)

select.shape

(1463, 7)

#### COMBINACIONES DE 2 ---- 3 COSECHAS

In [9]:
data['PERIODO'].value_counts()

202101    235
202012    230
202011    223
Name: PERIODO, dtype: int64

In [10]:
data['ZONAL'].value_counts()

BEP ZONAL 1    283
BEP ZONAL 2    216
BEP ZONAL 3    189
Name: ZONAL, dtype: int64

In [11]:
data['SEGMENTO'].value_counts()

S2    688
Name: SEGMENTO, dtype: int64

In [12]:
#[col for col in data.columns if 'NORMAL' in col.upper()]

In [13]:
data['NO_CLASIF_NORMAL'] = data['IS_NORMAL_IBK_SBS'].apply(lambda _: not(_)).astype(int)

pd.crosstab(
    data['NO_CLASIF_NORMAL'], data['IS_NORMAL_IBK_SBS']
)

IS_NORMAL_IBK_SBS,0,1
NO_CLASIF_NORMAL,,
0,0,678
1,10,0


In [14]:
for colum in ['VAR_1', 'VAR_2', 'VAR_3']:
    try:
        select[colum] = select[colum].apply(
            lambda _: 'NO_CLASIF_NORMAL' if _ in ['CLASIF_SBS__DEFICIENTE', 'CLASIF_SBS__DUDOSO', 'CLASIF_SBS__P.POTENCIAL'] else _)
    except:
        pass
    
    try:
        select_2[colum] = select_2[colum].apply(
            lambda _: 'NO_CLASIF_NORMAL' if _ in ['CLASIF_SBS__DEFICIENTE', 'CLASIF_SBS__DUDOSO', 'CLASIF_SBS__P.POTENCIAL'] else _)  
    except:
        pass

In [15]:
[col for col in data.columns if 'REFI' in col.upper()]

['TIENE_DEUDA_REFINANCIADO_RCC',
 'MTO_DEUDA_REFINANCIADO_COLDIRECT_SIN_REATCIVA_SF',
 'DEBT_REFINANCIADO_PERCENT_ULT_RCC',
 'TIENE_DEUDA_REFINANCIADO_IBK']

In [16]:
print([col for col in data.columns if 'VENCID' in col.upper()])

['_0.001_cut_up_DEBT_VENCIDO_PERCENT_ULT_RCC', 'TIENE_DEUDA_LEASING_VENCIDO_RCC', 'DEBT_FACTORING_VENCIDO_PERCENT_ULT_RCC', 'TIENE_DEUDA_VENCIDO_RCC', 'TIENE_VIEGNTE_VENCIDO_IBK', 'DEBT_LEASING_VENCIDO_PERCENT_ULT_RCC', 'DEBT_COMEX_VENCIDO_PERCENT_ULT_RCC', 'TIENE_VENCIDO_Y_VIGENTES_SF', '_0_cut_up_DEBT_VENCIDO_PERCENT_ULT_RCC', 'NRO_PROD_DEUDA_VENCIDO_SF_ULT_RCC', 'TIENE_EN_OTRAENTIDAD_DEBT_FACTORING_VENCIDO_SF_ULT_RCC', 'FLG_VENCIDA', 'TIENE_EN_OTRAENTIDAD_DEBT_PRESTAMOS_VENCIDO_SF_ULT_RCC', 'TIENE_DEUDA_SOBREGIROS__AVANCES_VENCIDO_RCC', 'TIENE_DEUDA_COMEX_VENCIDO_RCC', 'TIENE_DEUDA_VENCIDO_IBK', 'PERCENT_DEBT_VENCIDO_S_DTOTAL_SF_ULT_RCC', 'TIENE_EN_OTRAENTIDAD_DEBT_SOBREGIROS__AVANCES_VENCIDO_SF_ULT_RCC', 'TIENE_EN_OTRAENTIDAD_DEBT_COMEX_VENCIDO_SF_ULT_RCC', 'DEBT_PRESTAMOS_VENCIDO_PERCENT_ULT_RCC', 'DEBT_VENCIDO_PERCENT_ULT_RCC', 'NRO_PROD_OTRA_ENTIDAD_DEUDAS_VENCIDO_SF_ULT_RCC', 'TIENE_EN_OTRAENTIDAD_DEBT_DESCUENTOS_VENCIDO_SF_ULT_RCC', 'TIENE_DEUDA_DESCUENTOS_VENCIDO_RCC', 'TIENE

In [17]:
data['TIENE_DEUDA_REFINANCIADO_RCC'].value_counts(dropna=False)

0.0    688
Name: TIENE_DEUDA_REFINANCIADO_RCC, dtype: int64

In [18]:
[col for col in data.columns if 'fusion' in col.lower()]

['target_fusion_atraso_feve_3',
 'target_FINAL_fusion_atraso_feve_2Mbueno',
 'target_fusion_atraso_15_feve_all',
 'target_fusion_atraso_feve_3_3Mbueno',
 'target_fusion_atraso_feve',
 'target_fusion_atraso_feve_3_1Mbueno',
 'fusion_atraso_grantia',
 'target_fusion_atraso_15_feve_all_2Mbueno',
 'target_fusion_atraso_feve_2']

In [19]:
for row in select.sort_values(by=['SCORE'], ascending=False).iterrows():
    #print(type(row), row[0], row[1])
    print("/"*10)
    cols_1 = row[1][0]
    cols_2 = row[1][1]
    cols_3 = row[1][2]
    cantidad = row[1][3]
    correctos = row[1][4]
    score_feve = float(row[1][5])
    print("---> COLS: ", cols_1, cols_2, cols_3)
    print("---> CANTIDAD: ", cantidad, correctos, score_feve)
    
    if pd.isnull(cols_3):
        break

//////////
---> COLS:  _1_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC _aux_CASHOUT_CAIDA_40PERCENT_PAGOS_MASIVOS__U6M nan
---> CANTIDAD:  7 7 1.0


In [20]:
select.shape

(1463, 7)

In [21]:
data.shape, #data

((688, 2416),)

In [22]:
grupo_alerta = {}
cols_usada = []
lista_coinciden_valida = []
lista_alertados = []
listado_por_alerta = []

for row in select.sort_values(by=['SCORE'], ascending=False).iterrows():
    cols_1 = row[1][0]
    cols_2 = row[1][1]
    cols_3 = row[1][2]
    cantidad = row[1][3]
    correctos = row[1][4]
    score_feve = float(row[1][5])
    
    _all = (cols_1 + ' ' + cols_2  if pd.isnull(cols_3) else cols_1 + ' ' + cols_2 + ' ' + cols_3).lower()
    
    if cols_1 not in cols_usada and cols_2 not in cols_usada and cols_3 not in cols_usada: #and 'eptile_cut' not in _all \
       #and 'suspendido' not in _all and 'cancelado' not in _all and 'norte_exp' not in _all  and 'vencid' not in _all and 'cef' not in _all \
       #and 'tiene_trx' not in _all   and 'tiene_cashout' not in _all:
        try: 
            if pd.isnull(cols_3):
                data_que_cumple = data[
                    (data[cols_1] >= 1) & (data[cols_2] >= 1)
                ]
            else:
                cols_usada.append(cols_3)
                data_que_cumple = data[
                    (data[cols_1] >= 1) & (data[cols_2] >= 1) &  (data[cols_3] >= 1)
                ]
                
            alertados = set(data_que_cumple['COD_UNICO'])
            listado_por_alerta.append(list(alertados))
            cols_usada.append(cols_1)
            cols_usada.append(cols_2)
            
            
            bien_alertados = alertados & set(malos['COD_UNICO'])
            #print(data_que_cumple.shape, bien_alertados)
            if len(bien_alertados) > 3  and len(bien_alertados) / len(alertados) >= 0.1 and len(alertados) < 100:
                lista_alertados.extend(alertados)
                lista_coinciden_valida.extend(bien_alertados)
            
                print(cols_1, '|||', cols_2,  '|||', cols_3, 'de: ', len(set(alertados)), 'coinciden : ', len(set(bien_alertados)),
                      len(set(bien_alertados)) / len(set(alertados))
                )
                grupo_alerta[cols_1 + '|'+ cols_2 + '|'+ str(cols_3)] = 'de: {} coinciden: {}'.format(len(alertados), len(bien_alertados))
            
            if len(set(lista_coinciden_valida)) / len(set(malos['COD_UNICO'])) > 0.7:
                print("="*100)
                print("De {} alertó bien a: {}".format(
                    len(set(lista_alertados)), len(set(lista_coinciden_valida))
                ))
                
                clientes_detectados_ok = set(lista_coinciden_valida)
                clientes_no_alertar = set(lista_alertados) - set(clientes_detectados_ok)
                
                dicc_cliente_no_alertar = {}
                print("ANALIZANDO CLIENTES ")
                for cliente in clientes_no_alertar:
                    aparazco = 0
                    for lista_de_alerta in listado_por_alerta:
                        if cliente in lista_de_alerta:
                            aparazco += 1
                    dicc_cliente_no_alertar[cliente] = aparazco
                    
                dicc_cliente_ok = {}
                print("ANALIZANDO OK ")
                for cliente in clientes_detectados_ok:
                    aparazco = 0
                    for lista_de_alerta in listado_por_alerta:
                        if cliente in lista_de_alerta:
                            aparazco += 1
                    dicc_cliente_ok[cliente] = aparazco
                break
        
        except Exception as e:
            pass
            #print(">"*50, str(e))
        #    print(str(e))
        #    #coinciden_feve_duro

print(
    "SOLO CUBRE: ", len(set(lista_coinciden_valida)) / len(set(malos['COD_UNICO'])), len(set(lista_coinciden_valida)), 
     len(set(malos['COD_UNICO']))
)

sei_preocupa ||| _2_cut_up_DTOTAL_S_EBITDA_CEF_SITUACION ||| nan de:  7 coinciden :  4 0.5714285714285714
_0_cut_up_TIENE_TENDENCIA_CRECIENTE_{}_U7M_y.1 ||| TRX_CAIDA_30PERCENT_MES_ANTERIOR ||| _0.3_cut_up_ROTACION_CTASXCOBRAR_VAR_CEF_SITUACION de:  10 coinciden :  4 0.4
SOLO CUBRE:  0.19444444444444445 7 36
